In [2]:
pip install langchain openai gradio chromadb tiktoken

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/87/8b/d859c3125fc27e678f7da3625894d3d84e6599ef85f1559c1fc5b9c913ad/langchain-0.1.8-py3-none-any.whl.metadata
  Using cached langchain-0.1.8-py3-none-any.whl.metadata (13 kB)
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/26/a1/75474477af2a1dae3a25f80b72bbaf20e8296191ece7fff2f67984206f33/openai-1.12.0-py3-none-any.whl.metadata
  Using cached openai-1.12.0-py3-none-any.whl.metadata (18 kB)
  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/45/60/84e432d81d47a6f30603317eb782cc1024d47b90ef704168be99b931e9bb/gradio-4.19.2-py3-none-any.whl.metadata
  Using cached gradio-4.19.2-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/77/bb/5819ad3d7a3f581c4790c69d10757985c088020ae547be28b1553220c97a/chromadb-0.4.23-py3-none-any.whl.metadata
  Using 

  error: subprocess-exited-with-error
  
  × Building wheel for chroma-hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for chroma-hnswlib
ERROR: Could not build wheels for chroma-hnswlib, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import sys
import openai

from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory

import chromadb
import chromadb.config

import tiktoken
import gradio as gr

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
PERSIST = False

In [ ]:
query = None


def main_func(message , history):
  chat_history = []
  if PERSIST and os.path.exists("persist"):
    print("Reusing index...\n")
    vectorstore = Chroma(persist_directory="persist", embedding_function=OpenAIEmbeddings())
    index = VectorStoreIndexWrapper(vectorstore=vectorstore)
  else:
    loader = TextLoader("data.txt")
    if PERSIST:
      index = VectorstoreIndexCreator(vectorstore_kwargs={"persist_directory":"persist"}).from_loaders([loader])
    else:
      index = VectorstoreIndexCreator().from_loaders([loader])
  print(index)

  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
  chain = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(), retriever=index.vectorstore.as_retriever(), memory=memory, verbose=True)

  query = message
  result = chain({"question": query, "chat_history": chat_history})
  print(result['answer'])
  chat_history.append((query, result['answer']))
  return result['answer']

gr.ChatInterface(main_func ,title="CHIT CHAT 🤖",
                 chatbot=gr.Chatbot(height=550),
                 textbox=gr.Textbox(placeholder="Ask me anything about the artifacts" ),

                 ).launch())
